<a href="https://colab.research.google.com/github/bspoloo/SIS420-012024/blob/main/Laboratorios/Laboratorio%205/Laboratorio_5_(Usando_Pytorch).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio 5(Redes neuronales usando Pytorch) Grupo 1
 <h3>Implementacion del modelo redes neuronales con el framework de Pytorch para realizar predicciones de clasificacion.<h3>
 <HR>
 <h3>
  NOMBRE: POLO ORELLANA BRAYAN SIMON <br>
  CARRERA: INGENIERIA DE SISTEMAS <BR>
  FECHA: 02/05/2024 <BR>

  * [Enlace de invitacion para ser colaborador](https://github.com/bspoloo/SIS420-012024/invitations)
  
  * [Enlace al git hub](https://github.com/bspoloo/SIS420-012024/tree/main/Laboratorios/Laboratorio%205)
  
  * [Enlace al Colab](https://colab.research.google.com/drive/1XkWP1OWgmBz80uPD4q0tmch999rMs9S7?authuser=2#scrollTo=x-bqRMgg7f2v&uniqifier=1)
 <h3>

Para el laboratorio hizo uso del dataset `csgo_roun_snapshots.csv` para entrenar el modelo de **Redes neuronales** usando la libreria de **pythorch** y si en una partida de counter strike el equipo ganador entre **terrorista** y **antiterrorista**.

El enlace al dataset es [CS:GO Round Winner Classification](https://www.kaggle.com/datasets/christianlillelund/csgo-round-winner-classification).

El archivo `csgo_roun_snapshots.csv` cuenta con un conjunto de datos consta de instantáneas de rondas de aproximadamente 700 demostraciones de torneos de alto nivel en 2019 y 2020. Se filtraron las rondas de calentamiento y los reinicios, y para las rondas en vivo restantes se registró una instantánea de la ronda cada 20 segundos hasta que se decide la ronda..

Importamos la librerias necesarias:

In [ ]:
# Used for directory and path manipulation
import os
# Scientific and vector computation for Python
import numpy as np
# Library for graphics
from matplotlib import pyplot as plt
#Para separa el 20% y 80%
from sklearn.model_selection import train_test_split
import pandas as pd

import torch
from torch import optim  # Para optimizadores como SGD, Adam, etc.
from torch import nn  # Todos los módulos de redes neuronales
from torch.utils.data import DataLoader  # Facilita la gestión de conjuntos de datos creando mini lotes, etc.

import torchvision  # Paquete torch para cosas relacionadas con la visión
import torchvision.transforms as transforms  # Transformaciones que podemos realizar en nuestro conjunto de datos para aumentar
import torch.nn.functional as F  # Funciones sin parámetros, como (algunas) funciones de activación
import torchvision.datasets as datasets  # Conjuntos de datos estándar
from tqdm import tqdm  # ¡Para una barra de progreso agradable!
from torch.utils.data import Dataset

# Le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Red neuronal (Pytorch)

### Cargamos los datos del dataset
cargamos los datos haciendo el uso de la libreria **Pandas** que  es una herramienta poderosa y versátil utilizada para manipulación y análisis de datos. Ofrece estructuras de datos flexibles y eficientes para trabajar con datos tabulares, como hojas de cálculo en Excel o tablas SQL. Algunas de las funcionalidades clave de pandas incluyen:

In [ ]:
dataset = pd.read_csv('/content/gdrive/MyDrive/Datasets/csgo_round_snapshots.csv', delimiter=',')

# dataset = pd.read_csv('csgo_round_snapshots.csv', delimiter=',')

#para usarlo a travez de google colab
# dataset = dataset.sample(n=20000, random_state=42)

dataset

,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,...,t_grenade_flashbang,ct_grenade_smokegrenade,t_grenade_smokegrenade,ct_grenade_incendiarygrenade,t_grenade_incendiarygrenade,ct_grenade_molotovgrenade,t_grenade_molotovgrenade,ct_grenade_decoygrenade,t_grenade_decoygrenade,round_winner
0,175.00,0.0,0.0,de_dust2,False,500.0,500.0,0.0,0.0,4000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
1,156.03,0.0,0.0,de_dust2,False,500.0,500.0,400.0,300.0,600.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
2,96.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
3,76.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
4,174.97,1.0,0.0,de_dust2,False,500.0,500.0,192.0,0.0,18350.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122405,15.41,11.0,14.0,de_train,True,200.0,242.0,195.0,359.0,100.0,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,T
122406,174.93,11.0,15.0,de_train,False,500.0,500.0,95.0,175.0,11500.0,...,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,T
122407,114.93,11.0,15.0,de_train,False,500.0,500.0,495.0,475.0,1200.0,...,4.0,3.0,5.0,1.0,0.0,0.0,5.0,0.0,0.0,T
122408,94.93,11.0,15.0,de_train,False,500.0,500.0,495.0,475.0,1200.0,...,5.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,T


### Analisis del dataset

para tener una mejor vista del los tipos de datos del dataset, hacemos un `info()`:

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122410 entries, 0 to 122409
Data columns (total 97 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   time_left                     122410 non-null  float64
 1   ct_score                      122410 non-null  float64
 2   t_score                       122410 non-null  float64
 3   map                           122410 non-null  object 
 4   bomb_planted                  122410 non-null  bool   
 5   ct_health                     122410 non-null  float64
 6   t_health                      122410 non-null  float64
 7   ct_armor                      122410 non-null  float64
 8   t_armor                       122410 non-null  float64
 9   ct_money                      122410 non-null  float64
 10  t_money                       122410 non-null  float64
 11  ct_helmets                    122410 non-null  float64
 12  t_helmets                     122410 non-nul

Se puede observar que hay columnas con datos que no son numeros, por lo tanto se procede a tratarlos:

In [ ]:
#para contar cuantas clases contiene nuestra columna y
class_counts1 = dataset["map"].value_counts()
class_counts2 = dataset["bomb_planted"].value_counts()
class_counts3 = dataset["round_winner"].value_counts()

#mostramos la cantidad de clases tiene, y en que cantidad
class_counts1

map
de_inferno     23811
de_dust2       22144
de_nuke        19025
de_mirage      18576
de_overpass    14081
de_train       13491
de_vertigo     11137
de_cache         145
Name: count, dtype: int64

In [ ]:
class_counts2

bomb_planted
False    108726
True      13684
Name: count, dtype: int64

In [ ]:
class_counts3

round_winner
T     62406
CT    60004
Name: count, dtype: int64

podemo notar que todos son de tipo float, por lo cual no nos traera problemas al momento de entrenar el modelo.

cambiamos todos los string que a numeros:

In [ ]:
dataset['map'] = dataset['map'].map({'de_inferno': 0, 'de_dust2': 1, 'de_nuke': 2, 'de_mirage': 3, 'de_overpass': 4, 'de_train': 5, 'de_vertigo': 6, 'de_cache': 7})
dataset['bomb_planted'] = dataset['bomb_planted'].map({False: 0, True: 1})
dataset['round_winner'] = dataset['round_winner'].map({'T': 0, 'CT': 1})


Mostramos el dataset ya con todos los valores numericos:


In [ ]:
dataset

,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,...,t_grenade_flashbang,ct_grenade_smokegrenade,t_grenade_smokegrenade,ct_grenade_incendiarygrenade,t_grenade_incendiarygrenade,ct_grenade_molotovgrenade,t_grenade_molotovgrenade,ct_grenade_decoygrenade,t_grenade_decoygrenade,round_winner
0,175.00,0.0,0.0,1,0,500.0,500.0,0.0,0.0,4000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,156.03,0.0,0.0,1,0,500.0,500.0,400.0,300.0,600.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,96.03,0.0,0.0,1,0,391.0,400.0,294.0,200.0,750.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,76.03,0.0,0.0,1,0,391.0,400.0,294.0,200.0,750.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,174.97,1.0,0.0,1,0,500.0,500.0,192.0,0.0,18350.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122405,15.41,11.0,14.0,5,1,200.0,242.0,195.0,359.0,100.0,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
122406,174.93,11.0,15.0,5,0,500.0,500.0,95.0,175.0,11500.0,...,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
122407,114.93,11.0,15.0,5,0,500.0,500.0,495.0,475.0,1200.0,...,4.0,3.0,5.0,1.0,0.0,0.0,5.0,0.0,0.0,0
122408,94.93,11.0,15.0,5,0,500.0,500.0,495.0,475.0,1200.0,...,5.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0


### Separamos los datos de nuestra columna Y para cada clase:

In [ ]:
# hacemos uso del DataFrame llamado 'dataset' que contiene nuestros datos datos
# y es la columna que contiene las etiquetas de clase (en este caso, 'round_winner')

#creamos una variable temporal que contentra toda la columna de 'round_winner'
y_temp = dataset['round_winner']

# Para la clase 0

#donde y_temp es igual a 0, separamos los datos en train_class_0, test_class_0
data_class_0 = dataset[y_temp == 0]
train_class_0, test_class_0 = train_test_split(data_class_0, test_size=0.2, random_state=42)

# Para la clase 1

#donde y_temp es igual a 1, separamos los datos en train_class_1, test_class_1
data_class_1 = dataset[y_temp == 1]
train_class_1, test_class_1 = train_test_split(data_class_1, test_size=0.2, random_state=42)

In [ ]:
print(f"Para la clase 0 tenemos una cantidad de: { data_class_0.shape[0]} donde el 80% es: {train_class_0.shape[0]} y el 20% es: {test_class_0.shape[0]}")
print(f"Para la clase 1 tenemos una cantidad de: { data_class_1.shape[0]} donde el 80% es: {train_class_1.shape[0]} y el 20% es: {test_class_1.shape[0]}")
print(f"La cantidad total de datos es: {dataset.shape[0]}")

Para la clase 0 tenemos una cantidad de: 62406 donde el 80% es: 49924 y el 20% es: 12482
Para la clase 1 tenemos una cantidad de: 60004 donde el 80% es: 48003 y el 20% es: 12001
La cantidad total de datos es: 122410


### Separacion del 80% de los datos para entrenamiento y 20% para pruebas

Haremos uso de la libreria `sklearn` haciendo uso de su funcion `train_test_split()`, donde recibe como parametros:

`Arrays o matrices de características (X): `Estos son los datos que se utilizarán para hacer predicciones. Por lo general, son las variables independientes o características del conjunto de datos.

`test_size (opcional):` Este parámetro especifica el tamaño del conjunto de prueba. Puede ser un número decimal entre 0 y 1, que representa el porcentaje del conjunto de datos que se asignará al conjunto de prueba, o puede ser un entero que representa el número absoluto de muestras en el conjunto de prueba. Por ejemplo, si test_size=0.2, se asignará el 20% del conjunto de datos al conjunto de prueba.

`train_size (opcional):` Este parámetro especifica el tamaño del conjunto de entrenamiento. Al igual que test_size, puede ser un número decimal entre 0 y 1 o un entero que representa el número absoluto de muestras en el conjunto de entrenamiento. Si no se proporciona, se calcula automáticamente como 1 - test_size.

`random_state (opcional):` Este parámetro permite establecer una semilla para la generación de números pseudoaleatorios. Esto garantiza que la división de los datos sea reproducible. Si se establece en un número entero, el resultado será el mismo cada vez que se ejecute el código.

In [ ]:
#separando los datos de entrenamiento y pruebas

#para los datos de entrenamiento
train_dataset = pd.concat([train_class_0, train_class_1]).values
test_dataset = pd.concat([test_class_0, test_class_1]).values


indices_train = np.random.permutation(len(train_dataset))
indices_test = np.random.permutation(len(test_dataset))

train_dataset = pd.concat([train_class_0, train_class_1]).values

train_dataset = train_dataset[indices_train]
train_test = test_dataset[indices_test]

m_train = len(train_dataset)
m_test = len(test_dataset)

In [ ]:
print(m_train)
print(m_test)
print("El total de ejemplos es: ", (m_train + m_test))

97927
24483
El total de ejemplos es:  122410


#### Convertimos los datos a tensores de Pythorch

Este código conviertimos los datos **train_dataset** y **Test_dataset** en tensores de PyTorch.

Creamos la clase  indicar a PyTorch cuáles son los datos de entrada (features) y cuáles son las etiquetas (targets), creamnos una clase personalizada que herede de torch.utils.data.Dataset y sobrescribir los métodos **__len__** y **__getitem__.**

In [ ]:
class TuConjuntoDatos(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = torch.tensor(self.data[idx, :-1], dtype=torch.float32)  # Todas las columnas excepto la última
        target = torch.tensor(self.data[idx, -1], dtype=torch.long)  # Última columna (etiqueta)
        return features, target

    def get_features_shape(self):
        return (self.data.shape[1] - 1)

hacemos uso de la clase anterior, indicando cual es nuestra columna "target", en este caso la ultima, **-1**:

In [ ]:
train_dataset = TuConjuntoDatos(train_dataset, -1)  # No se necesitan etiquetas durante el entrenamiento
test_dataset = TuConjuntoDatos(test_dataset, -1)

In [ ]:
print(f"el 80% de datos es:{len(train_dataset)} y el 20% es: {len(test_dataset)}")
print( "la totalidad de datos es: ", dataset.shape[0])

el 80% de datos es:97927 y el 20% es: 24483
la totalidad de datos es:  122410


### Creacion de la clase de Red neuronal MLS

Aquí creamos nuestra red neuronal simple. Para más detalles aquí estamos subclasificando y
Heredando de **nn.Module**.

In [ ]:
class RedNeuronalMLS(nn.Module):
    def __init__(self, input_size, num_classes):
        super(RedNeuronalMLS, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = F.sigmoid(x)
        x = self.fc2(x)
        return x

### Definimos la arquitectura del modelo

En este apartado definiremos la cantidad de capas que tendra el modelo:

- **Capa de entrada:** Es la cantidad de caracteristicas con las que cuenta el dataset.
- **Capa de salida:** como este modelo cuenta con dos clases, por lo tanto solo tendra un valor de 2

In [ ]:
# Configurando parametros necesario
input_layer_size = train_dataset.get_features_shape() # Entrada de 96 caracteristicas, es la cantidad de columnas existente en el dataset
num_labels = 2         # como es una clasificacion tiene solo 2 salidas

print("la cantidad de elementos en la capa de entrada es: ",input_layer_size)
print("la cantidad de elementos en la capa de salida es: ",num_labels)

la cantidad de elementos en la capa de entrada es:  96
la cantidad de elementos en la capa de salida es:  2


#### Hiperparametros:

Hiperparámetros de nuestra red neuronal que dependen del conjunto de datos, y
También simplemente experimentando para ver qué funciona bien (tasa de aprendizaje, por ejemplo).

In [ ]:
input_size = input_layer_size
num_classes = num_labels
learning_rate = 0.001
batch_size = 1000
num_epochs = 3

print("la cantidad de elementos en la entrada es: ",input_size)

la cantidad de elementos en la entrada es:  96


#### Verificamos que el GPU este habilitado


Este código sirve para configurar el dispositivo en el que se ejecutarán los cálculos de PyTorch. Si una GPU está disponible en el sistema, el código configura el dispositivo para usar la GPU (CUDA), de lo contrario, se configura para utilizar la CPU.

In [ ]:
# Configuramos el dispositivo cuda para GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


#### Cargamos conjuntos de datos en lotes:

* **dataset:** El conjunto de datos que se va a cargar en lotes. En este caso, * * * train_dataset y test_dataset son los conjuntos de datos de entrenamiento y prueba, respectivamente.
* **batch_size:** El tamaño de los lotes de datos. Especifica cuántas muestras se cargarán a la vez durante el entrenamiento o la evaluación.
* **shuffle:** Un booleano que indica si se deben mezclar los datos antes de cargarlos en lotes. Esto es útil durante el entrenamiento para evitar que el modelo aprenda patrones basados en el orden de los datos.

In [ ]:
print(len(train_dataset))
print(len(test_dataset))

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

print("la cantidad de lotes para el dataset de entrenamiento es: ",len(train_loader))
print("la cantidad de lotes para el dataset de prueba es: ",len(test_loader))

97927
24483
la cantidad de lotes para el dataset de entrenamiento es:  98
la cantidad de lotes para el dataset de prueba es:  25


### Modelo de red neuronal

Este código crea una instancia de un modelo de red neuronal y lo mueve al dispositivo especificado (ya sea GPU o CPU) para su entrenamiento y evaluación.

* **RedNeuronalMLS:** Usamos la clase para crear el modelo.
* **input_size:** es el tamaño de entrada del modelo, es decir, el número de características en cada ejemplo de entrada.
num_classes es el número de clases en el problema de clasificación, es decir, el número de categorías diferentes a predecir.
* **.to(device):** mueve el modelo a la GPU (cuda) si está disponible, de lo contrario, lo deja en la CPU (cpu).

In [ ]:
# Initialize network
model = RedNeuronalMLS(input_size=input_size, num_classes=num_classes).to(device)

Mostramos algunas caracteristicas del modelo:

In [ ]:
model

RedNeuronalMLS(
  (fc1): Linear(in_features=96, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=2, bias=True)
)

#### Funcion de perdida o costo para el modelo:

como este es un modelo de clasificacion, se procede a crearlo con la funcion de **CrossEntropyLoss()** y el optimizador creando una instancia del **optimizador Adam**.

In [ ]:
# Perdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### Entrenamiento de la Red neuronal

Haciendo el uso dos ciclos For, en el primero para el uso del los **"epochs"** el cual ira iterando uno por uno dentro del segundo For, el c se recorre el cargador de datos de entrenamiento **(train_loader)** para obtener lotes **(batches)** de datos y sus respectivas etiquetas. **batch_idx **es el índice del lote actual, data contiene las entradas del lote y targets contiene las etiquetas correspondientes.

In [ ]:
# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):

        # Obtenemos el CUDA si es posible
        data = data.to(device=device)
        targets = targets.to(device=device)


        # Llegar a la forma correcta
        data = data.reshape(data.shape[0], -1)
        print(data.shape)
        # print("-"*30)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

  0%|          | 0/98 [00:00<?, ?it/s]

torch.Size([1000, 96])
torch.Size([1000, 96])
torch.Size([1000, 96])
torch.Size([1000, 96])


  5%|▌         | 5/98 [00:00<00:01, 46.64it/s]

torch.Size([1000, 96])
torch.Size([1000, 96])
torch.Size([1000, 96])
torch.Size([1000, 96])
torch.Size([1000, 96])


 11%|█         | 11/98 [00:00<00:01, 49.91it/s]

...

torch.Size([1000, 96])
torch.Size([1000, 96])
torch.Size([1000, 96])
torch.Size([1000, 96])
torch.Size([1000, 96])
torch.Size([1000, 96])
torch.Size([1000, 96])
torch.Size([1000, 96])
torch.Size([1000, 96])
torch.Size([1000, 96])
torch.Size([1000, 96])


 93%|█████████▎| 91/98 [00:01<00:00, 52.21it/s]

torch.Size([1000, 96])
torch.Size([1000, 96])
torch.Size([1000, 96])
torch.Size([1000, 96])
torch.Size([1000, 96])
torch.Size([1000, 96])
torch.Size([1000, 96])
torch.Size([1000, 96])
torch.Size([1000, 96])
torch.Size([1000, 96])
torch.Size([1000, 96])


100%|██████████| 98/98 [00:02<00:00, 46.53it/s]

torch.Size([1000, 96])
torch.Size([1000, 96])
torch.Size([1000, 96])
torch.Size([927, 96])


### validaciones

Para hacer las validaciones correspondientes, se **creo check_accuracy()** el cual recibe como parametros nuestros dataset_loader y el modelo, donde nos retornara el procentaje de precision y las predicciones hechas en un vector:

In [ ]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    predicciones = []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            predicciones.append(predictions)

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples, predicciones

#### Precision con los datos de entrenamiento:

In [ ]:
p_train, pred_train  = check_accuracy(train_loader, model)
print(f"Precisión en el conjunto de entrenamiento: {p_train*100:.2f} %")

Precisión en el conjunto de entrenamiento: 73.44 %


#### Precision con los datos de prueba:

In [ ]:
p_test, pred_test  = check_accuracy(test_loader, model)
print(f"Precisión en el conjunto de prueba: {p_test*100:.2f} %")

Precisión en el conjunto de prueba: 72.82 %


## ¿Concluciones?

Una vez hecha las predicciones del modelos, se puede determinar que el modelo reliza las predicciones con una precision del 72.82% con los datos de prueba y con un 73.44% con los datos de entrenamiento, puede considerarse casi buena, esto devido a que en un torneo de CS:GO las partidas son demasiado importantes, y tener el modelo con esta precision puede llegar a generar desconfianza.

### ¿Cuales son los mejores hiperparametros?

Se eligio aquellos hiperparametros de acuerdo a la grafica de costo realizada para cada modelo, guiandonos en que la grafica de costo no este muy curveada pero tampoco que este demasiado recta. A continuacion se muestra los hiperparametros seleccionados:

* **Capa de entrada:** Es la cantidad de caracteristicas con las que cuenta el dataset.
* **Capa de salida:** como este modelo cuenta con dos clases, por lo tanto solo tendra un valor de 2

* **learning_rate.-** el la tasa de aprendisaje, para este hiperparametro se definio con el valor de 0.001, porque cada vez que se disminuia el valor de este, la precision decaia en un 51.5%, pero si se mantenia se llegaba por encima del 60%.
* **batch_size.-** este hiperparametro nos indica la cantidad que tendra cada lote, se eligio el valor de 1000 guiandonos con la precision con los datos de entrenamiento y de prueba, si el valor era demasiado alto, como 10000 la precision rondaba entre los 61%, pero si lo reduciamos a 100 o 64, la precision llegaba a ser entre 51.6%, para una mejor precision se escogio el valor de 1000 por la preciosn que nos databa una precision de 72.82% y 73.44%.
* **num_epochs.-** para el numoero de epochs(epocas) se definio 3, esto porque se defnio de acuerdo a la precision y capacidad de computo, si se tenia un valor alto, el entrenamiento tardaba mas y la precision era pesima, pero si el valor era entre 1 o 2, el entrenamiento era muy rapido, pero la precision pesima, llegando a la conclucion de escoger el valor de 3, que nos daba una mejor precision y el tiempo de entrenamiento era demasiado corto.